# pre processing all current raw data for action id process
    steps:
    1 - **devide all files by user (with related information)**
    2 - merge each user's all related files
    3 - sort by timestamp
    4 - decide the certain "action key"(exp. 1:login 2:longoff)
    5 - break the log into action keys (origional using auto-generated non-repeat num ID with parsed log type)
    6 - new line while "logoff" (origional using block_id which not exist here)

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

file_dir ="./r5.2" #5.2 clip smaller for test, while using 6.2 instead for
LDAP_dir = "./r5.2/LDAP"
Processed_dir = "./pre_processed"

FILE_device = os.path.join(file_dir,"device.csv")
FILE_file = os.path.join(file_dir,"file.csv")
FILE_email = os.path.join(file_dir,"email.csv")
FILE_http = os.path.join(file_dir,"http.csv")
FILE_logon = os.path.join(file_dir,"logon.csv")
FILE_psychometric = os.path.join(file_dir,"psychometric.csv")

In [2]:
#load and deal with LDAP files
LDAP_files = []
for _, _, file in os.walk(LDAP_dir):
    LDAP_files.append(file)
LDAP_files = np.array(LDAP_files)
LDAP_files = np.squeeze(LDAP_files)

user_file = pd.read_csv(os.path.join(LDAP_dir, LDAP_files[0]))
print(user_file.shape)
print(user_file.head(5))

(1000, 9)
            employee_name  user_id                            email  \
0      Sara Indigo Berger  SIB0825      Sara.Indigo.Berger@dtaa.com   
1       Nerea Ingrid Good  NIG0433       Nerea.Ingrid.Good@dtaa.com   
2     Maggy Maryam Newman  MMN0579     Maggy.Maryam.Newman@dtaa.com   
3      Asher Lucas Daniel  ALD0521      Asher.Lucas.Daniel@dtaa.com   
4  Jael Phyllis Contreras  JPC0464  Jael.Phyllis.Contreras@dtaa.com   

                   role  business_unit             functional_unit  \
0              Salesman              1       5 - SalesAndMarketing   
1  ProductionLineWorker              1           3 - Manufacturing   
2              Salesman              1       5 - SalesAndMarketing   
3         Mathematician              1  2 - ResearchAndEngineering   
4              Salesman              1       5 - SalesAndMarketing   

     department               team                supervisor  
0     2 - Sales  5 - RegionalSales       Regan Kirestin Frye  
1  3 - Assembly 

In [3]:
#get all the non-repeate users in this dataset
for file in LDAP_files:
    temp = pd.read_csv(os.path.join(LDAP_dir, file))
    temp = temp[~temp.user_id.isin(user_file.user_id)]
    user_file = user_file.append(temp)
print(user_file.shape)


(1000, 9)


In [4]:
def show(file):
    print("shape is ",file.shape)

    print("\nindex and top 5 rows")
    print(file.head(5))


 step 1: split all files with their user id or other related information

In [2]:
# split by userid and email and save under different folder.
def split_and_save_by_column(df, column_name,suffix):
    value_list = set(df[column_name].tolist())
    # print(len(value_list))
    # name = value_list.pop()
    for name in tqdm(value_list):
        temp_df = df[(df[column_name] == name)]
        temp_out = Processed_dir+'/'+name
        if not os.path.exists(temp_out):
            os.mkdir(temp_out)
        temp_path = Processed_dir+'/'+name+'/'+suffix+".csv"
        temp_df.to_csv(temp_path, index=False)
    print("file "+suffix+" split and saved")
        # show(temp_df)

In [12]:
logon_file = pd.read_csv(FILE_logon)

show(logon_file)

shape is  (845351, 5)

index and top 5 rows
                         id                 date     user       pc activity
0  {Q9R3-U1PE27LQ-2164BJLT}  01/01/2010 06:20:00  SLW0616  PC-0110    Logon
1  {R1B6-S0LQ34EP-3361QPTV}  01/01/2010 06:28:00  RPM0600  PC-9164    Logon
2  {Y3R9-H0JY37RA-5902HXNX}  01/01/2010 07:04:00  DAM0170  PC-1525    Logon
3  {D5B1-I6BP26NF-3845HZNZ}  01/01/2010 07:23:00  WXW0044  PC-9422    Logon
4  {I0N6-I9PB69SZ-5272SLAT}  01/01/2010 07:31:00  RLD0349  PC-1290    Logon


In [16]:
logon_file = logon_file.drop(columns=["id", "pc"])
show(logon_file)

shape is  (845351, 3)

index and top 5 rows
                  date     user activity
0  01/01/2010 06:20:00  SLW0616    Logon
1  01/01/2010 06:28:00  RPM0600    Logon
2  01/01/2010 07:04:00  DAM0170    Logon
3  01/01/2010 07:23:00  WXW0044    Logon
4  01/01/2010 07:31:00  RLD0349    Logon


In [17]:
split_and_save_by_column(logon_file, "user","logon_file")

In [18]:
del logon_file

In [42]:
device_file = pd.read_csv(FILE_device)
show(device_file)

shape is  (407908, 5)

index and top 5 rows
                         id                 date     user       pc    activity
0  {R8N4-R8YE70QN-3287FCGL}  01/01/2010 06:46:40  RPM0600  PC-9164     Connect
1  {S2M7-P6FL85EG-7721KZFG}  01/01/2010 07:33:10  RPM0600  PC-9164  Disconnect
2  {T6E3-Z4ZI24WS-2427MCYM}  01/01/2010 07:59:59  RPM0600  PC-9164     Connect
3  {U7O7-D2ZJ25RP-9120NPLP}  01/01/2010 08:02:14  WXW0044  PC-9422     Connect
4  {F4W5-B9ID71RL-3407HBRZ}  01/01/2010 08:05:12  CSD0242  PC-8696     Connect


In [43]:
device_file = device_file.drop(columns=["id", "pc"])
show(device_file)

shape is  (407908, 3)

index and top 5 rows
                  date     user    activity
0  01/01/2010 06:46:40  RPM0600     Connect
1  01/01/2010 07:33:10  RPM0600  Disconnect
2  01/01/2010 07:59:59  RPM0600     Connect
3  01/01/2010 08:02:14  WXW0044     Connect
4  01/01/2010 08:05:12  CSD0242     Connect


In [44]:
split_and_save_by_column(device_file,"user", "device_file")

In [45]:
del device_file

In [39]:
email_file = pd.read_csv(FILE_email)
show(email_file)

shape is  (1975101, 7)

index and top 5 rows
                         id                 date  \
0  {D3T6-O8NM96KK-4228FZKP}  01/01/2010 07:27:08   
1  {R8H5-R2ZW22KL-0377UBLJ}  01/01/2010 07:38:27   
2  {N5A2-W5LE24AD-3983CXQJ}  01/01/2010 07:51:52   
3  {Q6U5-T5YU54VK-2335CAKA}  01/01/2010 08:03:06   
4  {D7N3-R3FO94ZV-8686YZYX}  01/01/2010 08:06:00   

                                                  to  \
0  Quinn.Kaden.Rojas@dtaa.com;Kendall.Petra.Burne...   
1  Ulysses.Marvin.Alvarado@dtaa.com;Adrian.Quenti...   
2                      Adena.Guinevere.Diaz@dtaa.com   
3  Sonia.Veda.Mclean@dtaa.com;Branden.Kirk.Aceved...   
4                      Katell.Fiona.Swanson@dtaa.com   

                             from   size  attachments  \
0    Dominic.Alvin.Moody@dtaa.com  22069            1   
1   William.Xerxes.Wiley@dtaa.com  23398            0   
2    Robert.Abdul.Weaver@dtaa.com  30045            0   
3  Doris.Wynne.Cervantes@dtaa.com  42409            0   
4    Courtney.Jena.G

In [40]:
email_file = email_file.drop(columns=["id","to","size","attachments","content"])
show(email_file)

shape is  (1975101, 2)

index and top 5 rows
                  date                            from
0  01/01/2010 07:27:08    Dominic.Alvin.Moody@dtaa.com
1  01/01/2010 07:38:27   William.Xerxes.Wiley@dtaa.com
2  01/01/2010 07:51:52    Robert.Abdul.Weaver@dtaa.com
3  01/01/2010 08:03:06  Doris.Wynne.Cervantes@dtaa.com
4  01/01/2010 08:06:00    Courtney.Jena.Garza@dtaa.com


In [41]:
split_and_save_by_column(email_file, "from", "email_file")
del email_file

In [ ]:
http_file = pd.read_csv(FILE_http)

In [4]:
show(http_file)

shape is  (23845079, 6)

index and top 5 rows
                         id                 date     user       pc  \
0  {2N3E-0D01II40-FFMF3150}  01/01/2010 06:28:41  RPM0600  PC-9164   
1  {6X2O-3U17ZL34-QLGR5009}  01/01/2010 06:30:19  SLW0616  PC-0110   
2  {9W5R-9N45VM21-ABBK9764}  01/01/2010 06:31:31  RPM0600  PC-9164   
3  {1H2R-5K40OD01-ESTK4218}  01/01/2010 06:34:58  RPM0600  PC-9164   
4  {7L8R-1M62XA85-QSOO5286}  01/01/2010 06:45:27  RPM0600  PC-9164   

                                                 url  \
0           http://facebook.com/Conatus/buridan.html   
1  http://target.com/1952_Winter_Olympics/holmenk...   
2   http://youtube.com/2004_World_Series/suppan.html   
3  http://youtube.com/2004_World_Series/varitek.html   
4    http://blogspot.com/Cyclura_nubila/cyclura.html   

                                             content  
0  paschal envying 1096 anselm sorcerer maternus ...  
1  meulan anna chantor 1096 psalter casabiancas g...  
2  meulan meulan anselm primi 1

In [5]:
http_file = http_file.drop(columns=["id","pc","url","content"])

In [6]:
http_file['action'] = "http"
show(http_file)

shape is  (23845079, 3)

index and top 5 rows
                  date     user action
0  01/01/2010 06:28:41  RPM0600   http
1  01/01/2010 06:30:19  SLW0616   http
2  01/01/2010 06:31:31  RPM0600   http
3  01/01/2010 06:34:58  RPM0600   http
4  01/01/2010 06:45:27  RPM0600   http


In [7]:
split_and_save_by_column(http_file, "user", "http_file")

NameError: name 'split_and_save_by_column' is not defined

In [11]:
file_file = pd.read_csv(FILE_file)
show(file_file)

shape is  (434769, 6)

index and top 5 rows
                         id                 date     user       pc  \
0  {M8R9-I5RI11PG-5467DBVR}  01/01/2010 06:51:00  RPM0600  PC-9164   
1  {K7D4-F3MJ16HT-2340NUGJ}  01/01/2010 08:09:28  CSD0242  PC-8696   
2  {B1G0-C3CC52KM-8002OOIA}  01/01/2010 08:17:52  CSD0242  PC-8696   
3  {D7F8-W4CG21DB-5694PJWH}  01/01/2010 08:18:33  WXW0044  PC-9422   
4  {B7Y6-R0HX46WQ-8863CNGR}  01/01/2010 08:20:17  CSD0242  PC-8696   

       filename                                            content  
0  5B9VCBIU.doc  D0-CF-11-E0-A1-B1-1A-E1 m45 adss arxiv 1128955...  
1  1I1WA4EL.txt  58-38-59-4B barbiger detmoldii labillardire 97...  
2  GSQEMB1R.doc  D0-CF-11-E0-A1-B1-1A-E1 rearers broody fattene...  
3  RDAOYBUK.txt  4B-4E-43-54 alboin salona turisindus audoin el...  
4  D92Z9FA8.pdf  25-50-44-46-2D wreckhouse till31122007 59 nia ...  


In [12]:
file_file = file_file.drop(columns=["id","pc", "filename","content"])

In [14]:
file_file['action'] = "file"
show(file_file)

shape is  (434769, 3)

index and top 5 rows
                  date     user action
0  01/01/2010 06:51:00  RPM0600   file
1  01/01/2010 08:09:28  CSD0242   file
2  01/01/2010 08:17:52  CSD0242   file
3  01/01/2010 08:18:33  WXW0044   file
4  01/01/2010 08:20:17  CSD0242   file


In [15]:
split_and_save_by_column(file_file,"user", "file_file")

100%|██████████| 219/219 [00:06<00:00, 32.27it/s]


file file_file split and saved


step 1 done
